In [6]:
import findspark
findspark.init()

In [7]:
import pyspark

from pyspark.sql import SparkSession

from pyspark.sql import functions as f

spark = SparkSession.builder.appName("PySpark").getOrCreate()

In [8]:
movies_smallDF = (spark.read.csv(
        path="D:\Term\HK7\BigData\movies_small.csv",
        sep=",",
        header=True,
        quote='"',
        schema="moviedId INT, title STRING, genres STRING",
    )
)

In [10]:
movies_smallDF.show()

+--------+--------------------+--------------------+
|moviedId|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)|  Adventure|Children|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      11|American Presiden...|Comedy|Drama|Romance|
|      12|Dracula: Dead and...|       Comedy|Horror|
|      13|        Balto (1995)|Adventure|Animati...|
|      14|        Nixon (1995)|               Drama|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      16|       Casino (1995)|         Crime|

In [11]:
#Question 1
movies_smallDF.withColumn("genres_array",f.split(f.col("genres"),"\|")).filter(f.array_contains(f.col('genres_array'),'Action')).select('moviedId','title','genres').show(5)

+--------+--------------------+--------------------+
|moviedId|               title|              genres|
+--------+--------------------+--------------------+
|       6|         Heat (1995)|Action|Crime|Thri...|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      20|  Money Train (1995)|Action|Comedy|Cri...|
+--------+--------------------+--------------------+
only showing top 5 rows



In [12]:
#Question 2
movies_smallDF.withColumn('count', f.size(f.split('genres', '\|'))).select('moviedId', 'title', 'count').sort('moviedId').show(5, truncate=False)

+--------+----------------------------------+-----+
|moviedId|title                             |count|
+--------+----------------------------------+-----+
|1       |Toy Story (1995)                  |5    |
|2       |Jumanji (1995)                    |3    |
|3       |Grumpier Old Men (1995)           |2    |
|4       |Waiting to Exhale (1995)          |3    |
|5       |Father of the Bride Part II (1995)|1    |
+--------+----------------------------------+-----+
only showing top 5 rows



In [20]:
movies_smallDF.withColumn('arr', f.split('genres', '\|')).withColumn('genres', f.explode('arr')).groupBy('genres').count().sort('count').show(5)

+------------------+-----+
|            genres|count|
+------------------+-----+
|(no genres listed)|   34|
|         Film-Noir|   87|
|              IMAX|  158|
|           Western|  167|
|           Musical|  334|
+------------------+-----+
only showing top 5 rows



In [14]:
#Question 4
movies_smallDF.withColumn('genres_array', f.split('genres', '\|')).withColumn('genres', f.explode('genres_array')).groupBy('genres').agg(f.collect_list("title").alias('moives_list')).show(truncate=100)

+------------------+----------------------------------------------------------------------------------------------------+
|            genres|                                                                                         moives_list|
+------------------+----------------------------------------------------------------------------------------------------+
|             Crime|[Heat (1995), Casino (1995), Money Train (1995), Get Shorty (1995), Copycat (1995), Assassins (19...|
|           Romance|[Grumpier Old Men (1995), Waiting to Exhale (1995), Sabrina (1995), American President, The (1995...|
|          Thriller|[Heat (1995), GoldenEye (1995), Money Train (1995), Get Shorty (1995), Copycat (1995), Assassins ...|
|         Adventure|[Toy Story (1995), Jumanji (1995), Tom and Huck (1995), GoldenEye (1995), Balto (1995), Cutthroat...|
|             Drama|[Waiting to Exhale (1995), American President, The (1995), Nixon (1995), Casino (1995), Sense and...|
|               War|[Ric

In [15]:
#Question 5
movies_smallDF.withColumn('arr', f.split('title', ' ')).withColumn('last_element', f.element_at('arr', -1)).withColumn('year', f.substring('last_element', 2,4).cast('int')).withColumn('arr_genres', f.split('genres', '\|')).withColumn('genres', f.explode('arr_genres')).groupBy('genres').agg(f.min('year').alias('first_appearance')).filter(f.substring('genres',0,10).isin(['Sci-Fi','Animation'])).select(f.col('genres').alias('genre'), 'first_appearance').show(1000, truncate=False)

+---------+----------------+
|genre    |first_appearance|
+---------+----------------+
|Animation|1908            |
|Sci-Fi   |1902            |
+---------+----------------+



In [32]:
#Question 6
moviesDF = spark.read.options(delimiter=",",header='True',inferSchema='True').csv("./BigData/movies_small.csv")
ratingsDF = spark.read.options(delimiter=",",header='True',inferSchema='True').csv("./BigData/ratings_small.csv")

In [33]:
joinDF = moviesDF.join(ratingsDF, on=['movieId'],how='inner')
joinDF.withColumn('genres_array',f.split('genres','\|')).withColumn('single_genres',f.explode('genres_array')).show()

joinDF.withColumn('genres_array',f.split('genres','\|')).withColumn('single_genres',f.explode('genres_array')).groupBy('single_genres').avg('rating').select('single_genres',f.col('avg(rating)').alias('Average rating')).show()

+-------+--------------------+--------------------+------+------+---------+--------------------+-------------+
|movieId|               title|              genres|userId|rating|timestamp|        genres_array|single_genres|
+-------+--------------------+--------------------+------+------+---------+--------------------+-------------+
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|[Adventure, Anima...|    Adventure|
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|[Adventure, Anima...|    Animation|
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|[Adventure, Anima...|     Children|
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|[Adventure, Anima...|       Comedy|
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|[Adventure, Anima...|      Fantasy|
|      3|Grumpier Old Men ...|      Comedy|Romance|     1|   4.0|964981247|   [Comedy, Romance]|       Comedy|
|